In [21]:
import asyncio
from data_project.gsheets import DateSheet
from datetime import date, timedelta, datetime
import pandas as pd


target_date = (date.today() - timedelta(days=1))

async def create_sheet():
    return DateSheet('[RP_CN] Revenue Report', date=target_date.isoformat())

task_sheet = asyncio.create_task(create_sheet())
await task_sheet
mysheet = task_sheet.result()

# Filter Data

In [22]:
import datetime

mysheet = DateSheet('[RP_CN] 不夜之城禮包收入 ', date=target_date.isoformat())
mysheet.change_sheet('DATA')
sheet = mysheet.worksheet

data = sheet.get_all_values()

headers = data[0]
data = data[1:]

date_col_index = headers.index("DATE")

date_values = [row[date_col_index] for row in data[0:]]

max_value = None
for value in date_values:
    try:
        num_value = datetime.date.fromisoformat(value)
        if max_value is None or num_value > max_value:
            max_value = num_value
    except ValueError:
        continue

if max_value is not None:
    print("max value:", max_value)
else:
    print("False")

/Users/serenayu/Documents/data-project/data_project/gsheets.py:81: UserWarning: No columns named '日期' or 'date'.
  warnings.warn("No columns named '日期' or 'date'.")
/Users/serenayu/Documents/data-project/data_project/gsheets.py:91: UserWarning: Given date 2024-09-07 not found, min 2023-12-31 max 2024-09-05
  warnings.warn(f'Given date {date} not found, min {dates[self.headers_row]} max {dates[-1]}')


max value: 2024-09-05


In [33]:
import datetime

target_date = (datetime.date.today() - datetime.timedelta(days=1))

async def create_sheet():
    return DateSheet('[RP_CN] Revenue Report', date=target_date.isoformat())

task_sheet = asyncio.create_task(create_sheet())
await task_sheet
mysheet = task_sheet.result()

# Change sheet
mysheet.change_sheet("data", headers_row=2)
sheet = mysheet.worksheet

# Yesterday
today = datetime.datetime.now()
yesterday = today - datetime.timedelta(days=1)
yesterday_str = yesterday.strftime('%Y-%m-%d')

# Get all values from sheet
data = sheet.get_all_values()
# headers are in the second row (index 1)
headers = data[1]
data = data[2:]

df = pd.DataFrame(data, columns=headers)

def str2date(x):
    try:
        return datetime.date.fromisoformat(x)
    except:
        return datetime.date(1900, 1, 1)

filtered_df = df[(df['分類1'] == '不夜之城') & (df['PUR Date'].apply(str2date) == max_value)]

print(filtered_df)

filtered_num_rows = filtered_df.shape[0]
print("Number of rows in the filtered DataFrame:", filtered_num_rows)

       serverPrefix      userId       Create Time Package ID Package Name  \
228507         1049  1049010887  2023-11-08 11:36       1761      日冕石特惠禮包   
228509         1049  1049010887  2023-11-08 11:36       1761      日冕石特惠禮包   
228510         1136  1136019490   2024-07-03 5:29       1761      日冕石特惠禮包   
228512         1104  1104007679  2024-03-15 20:25       1761      日冕石特惠禮包   
228548         1152  1152016530   2024-08-26 23:9       1776      日冕石特惠禮包   
228549         1152  1152016530   2024-08-26 23:9       1776      日冕石特惠禮包   

         Package Type price  Purchase Time   分類1   分類2     USD  \
228507  TreasureHouse   250  5/9/2024 2:21  不夜之城  活動禮包  $3.750   
228509  TreasureHouse   250  5/9/2024 2:21  不夜之城  活動禮包  $3.750   
228510  TreasureHouse   250  5/9/2024 2:22  不夜之城  活動禮包  $3.750   
228512  TreasureHouse   250  5/9/2024 2:24  不夜之城  活動禮包  $3.750   
228548  TreasureHouse   250  5/9/2024 2:59  不夜之城  活動禮包  $3.750   
228549  TreasureHouse   250  5/9/2024 2:59  不夜之城  活動禮包  $3.750  

In [66]:
filtered_df.columns

Index(['serverPrefix', 'userId', 'Create Time', 'Package ID', 'Package Name',
       'Package Type', 'price', 'Purchase Time', '分類1', '分類2', 'USD',
       'Unique ID', 'Reg Date', 'PUR Date', 'PUR Week', 'PUR MTH', 'Retention',
       'UNI by PUR', 'USD by Day', '白名單'],
      dtype='object')

# Insert Data

In [43]:
# select the target sheet
mysheet = DateSheet('[RP_CN] 不夜之城禮包收入 ', date=target_date.isoformat())
mysheet.change_sheet('DATA')
sheet = mysheet.worksheet
data = filtered_df.copy()
#data = data.iloc[: 10]  # only update the first 10 rows

# append empty row
num_columns = len(sheet.row_values(1))
empty_row = [''] * num_columns
sheet.append_row(empty_row)

# define the relation of columns
column_names = {
    "serverPrefix": "serverPrefix",
    "userId": "userId",
    "Purchase Time": "Purchase Time",
}

# rename the col of data
data.columns = [column_names.get(c, c).lower() for c in data.columns]

# sort & filter columns
for header in mysheet.headers:
    if header not in data.columns:
        data.loc[:, header] = None
data = data[mysheet.headers]

# update formula
row_num = len(sheet.col_values(1))
data.loc[:, '購買次數上限 by package_id'] = range(len(data))
def update_formula(x):
    r = row_num + 1 + x
    return f"=COUNTIFS(B:B,B{r},D:D,D{r})"
data.loc[:, "購買次數上限 by package_id"] = data.loc[:, "購買次數上限 by package_id"].apply(update_formula)

# sort rows
# data['ct'] = data['ct'].str.replace(" ", "T").apply(datetime.fromisoformat)  # convert str into datetime
data.sort_values("purchase time", ascending=True, inplace=True)

# compute the cell to paste
cell = f"A{row_num + 1}"

# update the target sheet
sheet.update(cell, data.fillna('').values.tolist(), raw=False)


/Users/serenayu/Documents/data-project/data_project/gsheets.py:81: UserWarning: No columns named '日期' or 'date'.
  warnings.warn("No columns named '日期' or 'date'.")
/var/folders/ct/v7_skn0d1qj1fx21m526kmwm0000gn/T/ipykernel_20816/546836252.py:45: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update(cell, data.fillna('').values.tolist(), raw=False)


{'spreadsheetId': '18uqZjmH4LHTrbJyp-G5uMJlWR6TmIsK187simq8VMYY',
 'updatedRange': 'DATA!A21258:J21263',
 'updatedRows': 6,
 'updatedColumns': 10,
 'updatedCells': 60}